# Install tpot only if not there

In [3]:
pip install tpot

     ---------------------------------------- 87.4/87.4 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 114.3/114.3 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 70.9/70.9 MB 7.0 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=b2b7491679265e7a8cdb3d073fed3ec97fe881d9dda1ad21692d058ea8d140fd
  Stored in directory: c:\users\deepti gupta\appdata\local\pip\cache\wheels\48\8c\93\3afb1916772591fe6bcc25cdf8b1c5bdc362f0ec8e2f0fd413
Successfully built stopit
Note: you may need to restart the kernel to use updated packages.


## Ignoring warnings

In [66]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Import packages

In [67]:
from tpot import TPOTClassifier
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import get_scorer_names, get_scorer
from datetime import datetime, timedelta
import nltk
import numpy as np

from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler

# Read comapny tickertape data tsv file created after webscraping

In [68]:
data = pd.read_csv('companyList.tsv', delimiter='\t',encoding='utf-8')

In [69]:
data.head()

,sNo,Company Name,Full Company Name,sc Performance,sc Valuation,sc Growth,sc Profitability,sc Entry Point,sc Red Flags,Income Statements,...,2020 PBT,2021 Total Revenue,2021 EBITDA,2021 Net Income,2021 PBT,2022 Total Revenue,2022 EBITDA,2022 Net Income,2022 PBT,PE Ratio
0,1,ABFRL,Aditya Birla Fashion and Retail Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is not in the overbought zone,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,-32.89,"5,322.32",655.29,-672.51,-837.54,"8,239.11","1,241.33",-108.72,-144.93,-522.45
1,2,AFFLE,Affle (India) Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,High Strong financials and growth story over ...,High Showing good signs of profitability & ef...,BadThe stock is overpriced and in the overboug...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,79.23,558.00,171.20,134.81,147.92,"1,153.34",284.33,213.90,244.81,55.23
2,3,AARTIDRUGS,Aarti Drugs Ltd,"Avg Price return has been average, nothing ex...",High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Good The stock is underpriced and is not in t...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,185.30,"2,162.57",445.12,280.41,369.01,"2,499.97",343.92,205.04,269.96,26.20
3,4,AARTIIND,Aarti Industries Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,676.18,"4,506.80",982.23,523.47,664.55,"7,000.76","1,929.61","1,307.19","1,526.80",33.66
4,5,ACC,ACC Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Lower than Industry Revenue GrowthA higher-tha...,...,"1,708.85","16,370.03","3,161.68","1,862.99","2,506.38","17,656.07","1,597.86",649.44,871.25,70.59


# Creating Sentiment based on score card and writing it to a file

In [70]:
# Step 1: Load and preprocess the data
data = pd.read_csv('companyList.tsv', delimiter='\t',encoding='utf-8')  # Assuming you have a CSV file with sentiment data


# Create a new column for sentiment labels
data['Entry_point_Sentiment'] = ''
data['Growth_Sentiment']= ''
data['Profitability_Sentiment']= ''

# Function to assign sentiment labels based on keywords
def assign_sentiment_label(text):
    lowercase_text = text.lower()
    if 'high' in lowercase_text:
        return 2
    elif 'avg' in lowercase_text:
        return 1
    else:
        return 0

def redflag_valuation_sentiment_label(text):
    lowercase_text = text.lower()
    if 'high' in lowercase_text:
        return 0
    elif 'avg' in lowercase_text:
        return 1
    else:
        return 2

def entrypoint_sentiment_label(text):
    lowercase_text = text.lower()
    if 'bad' in lowercase_text:
        return 0
    elif 'good' in lowercase_text:
        return 2
    else:
        return 1
data['Entry_point_Sentiment'] = data['sc Entry Point'].apply(entrypoint_sentiment_label)
data['Growth_Sentiment'] = data['sc Growth'].apply(assign_sentiment_label)
data['Profitability_Sentiment'] = data['sc Profitability'].apply(assign_sentiment_label)
data['RedFlag_Sentiment'] = data['sc Red Flags'].apply(redflag_valuation_sentiment_label)
data['Performance_Sentiment'] = data['sc Performance'].apply(assign_sentiment_label)
data['Valuation_Sentiment'] = data['sc Valuation'].apply(redflag_valuation_sentiment_label)


# Save the updated dataset
data.to_csv('companyList_sentiment.csv', index=False)

In [71]:
data.head()

,sNo,Company Name,Full Company Name,sc Performance,sc Valuation,sc Growth,sc Profitability,sc Entry Point,sc Red Flags,Income Statements,...,2022 EBITDA,2022 Net Income,2022 PBT,PE Ratio,Entry_point_Sentiment,Growth_Sentiment,Profitability_Sentiment,RedFlag_Sentiment,Performance_Sentiment,Valuation_Sentiment
0,1,ABFRL,Aditya Birla Fashion and Retail Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is not in the overbought zone,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,"1,241.33",-108.72,-144.93,-522.45,1,0,2,2,0,0
1,2,AFFLE,Affle (India) Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,High Strong financials and growth story over ...,High Showing good signs of profitability & ef...,BadThe stock is overpriced and in the overboug...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,284.33,213.90,244.81,55.23,0,2,2,2,0,0
2,3,AARTIDRUGS,Aarti Drugs Ltd,"Avg Price return has been average, nothing ex...",High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Good The stock is underpriced and is not in t...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,343.92,205.04,269.96,26.20,2,0,2,2,1,0
3,4,AARTIIND,Aarti Industries Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,"1,929.61","1,307.19","1,526.80",33.66,1,0,2,2,0,0
4,5,ACC,ACC Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Lower than Industry Revenue GrowthA higher-tha...,...,"1,597.86",649.44,871.25,70.59,1,0,2,2,0,0


# Creating Avg Sentiment based on all scorecards

In [72]:
data['Sentiment']=(data['Entry_point_Sentiment']+data['Growth_Sentiment']+data['Profitability_Sentiment']+data['Performance_Sentiment']+data['RedFlag_Sentiment']+data['Valuation_Sentiment'])/6
print(data['Sentiment'])
data.to_csv('companyList_avg_sentiment.csv', index=False)

0      0.833333
1      1.000000
2      1.166667
3      0.833333
4      0.833333
         ...   
492    1.000000
493    0.833333
494    0.833333
495    0.833333
496    0.333333
Name: Sentiment, Length: 497, dtype: float64


# Removing special characters & cleaning data

In [73]:
data = data.replace("NAN", 0)
data = data.fillna(0)

In [74]:
data = pd.read_csv('companyList_avg_sentiment.csv')
data.head()
def remove_special_characters(text):
    # Remove special characters and punctuation using regular expressions
    cleaned_text = re.sub(r'[^\d\s.-]', '', text)
    return cleaned_text

data['2019 Total Revenue']= data['2019 Total Revenue'].apply(remove_special_characters)
data['2019 EBITDA']= data['2019 EBITDA'].apply(remove_special_characters)
data['2019 Net Income']= data['2019 Net Income'].apply(remove_special_characters)
data['2019 PBT']= data['2019 PBT'].apply(remove_special_characters)
data['2020 Total Revenue']= data['2020 Total Revenue'].apply(remove_special_characters)
data['2020 EBITDA']= data['2020 EBITDA'].apply(remove_special_characters)
data['2020 Net Income']= data['2020 Net Income'].apply(remove_special_characters)
data['2020 PBT']= data['2020 PBT'].apply(remove_special_characters)
data['2021 Total Revenue']= data['2021 Total Revenue'].apply(remove_special_characters)
data['2021 EBITDA']= data['2021 EBITDA'].apply(remove_special_characters)
data['2021 Net Income']= data['2021 Net Income'].apply(remove_special_characters)
data['2021 PBT']= data['2021 PBT'].apply(remove_special_characters)
data['2022 Total Revenue']= data['2022 Total Revenue'].apply(remove_special_characters)
data['2022 EBITDA']= data['2022 EBITDA'].apply(remove_special_characters)
data['2022 Net Income']= data['2022 Net Income'].apply(remove_special_characters)
data['2022 PBT']= data['2022 PBT'].apply(remove_special_characters)
data['PE Ratio']= data['PE Ratio'].apply(remove_special_characters)




data.to_csv('companyList_avg_sentiment_clean.csv', index=False)

In [75]:
newsdata = pd.read_csv('companyNewsList.tsv', delimiter='\t',encoding='utf-8')

In [76]:
newsdata.head()

,sNo,Company Name,Full Company Name,News Title,News Article,News Summary,NewsDate
0,4,AARTIIND,Aarti Industries Ltd,Trade setup for Wednesday: Top 15 things to kn...,The recovery in late trade amid consolidation ...,"This was followed by 18,800 strike comprising ...",5 days ago
1,5,ACC,ACC Ltd,"Who Is Nitin Agarwal, New DG Of BSF And Kerala...",After lying vacant for five months post the re...,After lying vacant for five months post the re...,12 hours ago
2,5,ACC,ACC Ltd,India-Pakistan World Cup match in Ahmedabad a ...,The 'hybrid' Asia Cup model is likely to be ap...,"""Oman Cricket board chief Pankaj Khimji, one o...",1 day ago
3,5,ACC,ACC Ltd,ACC Likely To Go Ahead With PCB’s Hybrid Model...,Asia Cup trophy. | (Credits: Twitter)\n\nThe A...,Asia Cup trophy.\n| (Credits: Twitter)The Asia...,1 day ago
4,6,ADANIGREEN,Adani Green Energy Ltd,NSE revises price bands on three Adani Group c...,National Stock Exchange (NSE) has revised pric...,National Stock Exchange (NSE) has revised pric...,5 days ago


In [77]:
def convert_relative_time(relative_time):
    pattern = r'(\d+)\s+(\w+)\s+ago'
    match = re.search(pattern, relative_time)
    if match:
        count = int(match.group(1))
        unit = match.group(2)
        if unit == 'days':
            return timedelta(days=count)
        elif unit == 'hours':
            return timedelta(hours=count)
        elif unit == 'minutes':
            return timedelta(minutes=count)
    return timedelta()

In [78]:
newsdata['NewsDate']= datetime.now() - newsdata['NewsDate'].apply(convert_relative_time)

In [80]:

sia = SentimentIntensityAnalyzer()

filtered_news = newsdata[newsdata['NewsDate'] > pd.Timestamp.now() - pd.Timedelta(days=7)]
#filtered_news = newsdata[newsdata['NewsDate'] > pd.Timestamp.now() - pd.Timedelta]
filtered_news['News Sentiment Score'] = filtered_news['News Article'].astype(str).apply(lambda x: sia.polarity_scores(x)['compound'])
avg_sentiment = filtered_news.groupby('Company Name')['News Sentiment Score'].mean()

In [81]:
avg_sentiment.to_frame()

,News Sentiment Score
Company Name,
AARTIIND,0.998200
ABB,0.990500
ABBOTINDIA,0.983100
ACC,0.960767
ADANIENT,0.429400
...,...
VOLTAS,0.983100
WIPRO,0.985233
YESBANK,0.664400


In [83]:
data.head()

,sNo,Company Name,Full Company Name,sc Performance,sc Valuation,sc Growth,sc Profitability,sc Entry Point,sc Red Flags,Income Statements,...,2022 Net Income,2022 PBT,PE Ratio,Entry_point_Sentiment,Growth_Sentiment,Profitability_Sentiment,RedFlag_Sentiment,Performance_Sentiment,Valuation_Sentiment,Sentiment
0,1,ABFRL,Aditya Birla Fashion and Retail Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is not in the overbought zone,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,-108.72,-144.93,-522.45,1,0,2,2,0,0,0.833333
1,2,AFFLE,Affle (India) Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,High Strong financials and growth story over ...,High Showing good signs of profitability & ef...,BadThe stock is overpriced and in the overboug...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,213.90,244.81,55.23,0,2,2,2,0,0,1.000000
2,3,AARTIDRUGS,Aarti Drugs Ltd,"Avg Price return has been average, nothing ex...",High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Good The stock is underpriced and is not in t...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,205.04,269.96,26.20,2,0,2,2,1,0,1.166667
3,4,AARTIIND,Aarti Industries Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,1307.19,1526.80,33.66,1,0,2,2,0,0,0.833333
4,5,ACC,ACC Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Lower than Industry Revenue GrowthA higher-tha...,...,649.44,871.25,70.59,1,0,2,2,0,0,0.833333


In [84]:
newsdata.head()

,sNo,Company Name,Full Company Name,News Title,News Article,News Summary,NewsDate
0,4,AARTIIND,Aarti Industries Ltd,Trade setup for Wednesday: Top 15 things to kn...,The recovery in late trade amid consolidation ...,"This was followed by 18,800 strike comprising ...",2023-06-08 12:37:37.872495
1,5,ACC,ACC Ltd,"Who Is Nitin Agarwal, New DG Of BSF And Kerala...",After lying vacant for five months post the re...,After lying vacant for five months post the re...,2023-06-13 00:37:37.872495
2,5,ACC,ACC Ltd,India-Pakistan World Cup match in Ahmedabad a ...,The 'hybrid' Asia Cup model is likely to be ap...,"""Oman Cricket board chief Pankaj Khimji, one o...",2023-06-13 12:37:37.872495
3,5,ACC,ACC Ltd,ACC Likely To Go Ahead With PCB’s Hybrid Model...,Asia Cup trophy. | (Credits: Twitter)\n\nThe A...,Asia Cup trophy.\n| (Credits: Twitter)The Asia...,2023-06-13 12:37:37.872495
4,6,ADANIGREEN,Adani Green Energy Ltd,NSE revises price bands on three Adani Group c...,National Stock Exchange (NSE) has revised pric...,National Stock Exchange (NSE) has revised pric...,2023-06-08 12:37:37.872495


In [85]:
combined_data = data.merge(avg_sentiment, on='Company Name', how='outer')

In [86]:
combined_data.head()

,sNo,Company Name,Full Company Name,sc Performance,sc Valuation,sc Growth,sc Profitability,sc Entry Point,sc Red Flags,Income Statements,...,2022 PBT,PE Ratio,Entry_point_Sentiment,Growth_Sentiment,Profitability_Sentiment,RedFlag_Sentiment,Performance_Sentiment,Valuation_Sentiment,Sentiment,News Sentiment Score
0,1.0,ABFRL,Aditya Birla Fashion and Retail Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is not in the overbought zone,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,-144.93,-522.45,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,NaN
1,2.0,AFFLE,Affle (India) Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,High Strong financials and growth story over ...,High Showing good signs of profitability & ef...,BadThe stock is overpriced and in the overboug...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,244.81,55.23,0.0,2.0,2.0,2.0,0.0,0.0,1.000000,NaN
2,3.0,AARTIDRUGS,Aarti Drugs Ltd,"Avg Price return has been average, nothing ex...",High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Good The stock is underpriced and is not in t...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,269.96,26.20,2.0,0.0,2.0,2.0,1.0,0.0,1.166667,NaN
3,4.0,AARTIIND,Aarti Industries Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,1526.80,33.66,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,0.998200
4,5.0,ACC,ACC Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Lower than Industry Revenue GrowthA higher-tha...,...,871.25,70.59,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,0.960767


In [87]:
combined_data = combined_data.fillna(0)
combined_data = combined_data.replace(r'^\s*$', 0, regex=True)

In [88]:
combined_data.head()

,sNo,Company Name,Full Company Name,sc Performance,sc Valuation,sc Growth,sc Profitability,sc Entry Point,sc Red Flags,Income Statements,...,2022 PBT,PE Ratio,Entry_point_Sentiment,Growth_Sentiment,Profitability_Sentiment,RedFlag_Sentiment,Performance_Sentiment,Valuation_Sentiment,Sentiment,News Sentiment Score
0,1.0,ABFRL,Aditya Birla Fashion and Retail Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is not in the overbought zone,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,-144.93,-522.45,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,0.000000
1,2.0,AFFLE,Affle (India) Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,High Strong financials and growth story over ...,High Showing good signs of profitability & ef...,BadThe stock is overpriced and in the overboug...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,244.81,55.23,0.0,2.0,2.0,2.0,0.0,0.0,1.000000,0.000000
2,3.0,AARTIDRUGS,Aarti Drugs Ltd,"Avg Price return has been average, nothing ex...",High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Good The stock is underpriced and is not in t...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,269.96,26.20,2.0,0.0,2.0,2.0,1.0,0.0,1.166667,0.000000
3,4.0,AARTIIND,Aarti Industries Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Higher than Industry Revenue GrowthA higher-th...,...,1526.80,33.66,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,0.998200
4,5.0,ACC,ACC Ltd,Low Hasn't faired well - amongst the low perf...,High Seems to be overvalued vs the market ave...,Low Lagging behind the market in financials g...,High Showing good signs of profitability & ef...,Avg The stock is overpriced but is not in the...,Low No red flag found,Lower than Industry Revenue GrowthA higher-tha...,...,871.25,70.59,1.0,0.0,2.0,2.0,0.0,0.0,0.833333,0.960767


In [89]:


X = combined_data[['News Sentiment Score','2019 Total Revenue','2019 EBITDA','2019 Net Income','2019 PBT','2020 Total Revenue','2020 EBITDA','2020 Net Income','2020 PBT','2021 Total Revenue','2021 EBITDA','2021 Net Income','2021 PBT','2022 Total Revenue','2022 EBITDA','2022 Net Income','2022 PBT','PE Ratio']]

# Get the values of X
X_values = X.values
print(X)
X.to_csv('X_values.csv', index=False)


     News Sentiment Score 2019 Total Revenue 2019 EBITDA 2019 Net Income  \
0                0.000000            8182.50      656.10          321.22   
1                0.000000             249.79       70.71           48.82   
2                0.000000            1567.12      217.09           89.75   
3                0.998200            4169.66      967.19          491.73   
4                0.960767           15990.00     2745.23         1377.41   
..                    ...                ...         ...             ...   
493              0.000000             402.83      219.61          153.73   
494              0.000000                  0           0               0   
495              0.000000             164.76       68.02            8.97   
496              0.000000                  0           0               0   
497              0.884067                  0           0               0   

    2019 PBT 2020 Total Revenue 2020 EBITDA 2020 Net Income 2020 PBT  \
0     149.10   

In [90]:
combined_data.to_csv('companyList_xvalues.csv', index=False)

In [91]:

# Create an instance of MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 2))
# Fit the scaler to the data and transform the values
combined_data['News Sentiment Score'] = scaler.fit_transform(combined_data[['News Sentiment Score']])
combined_data['2019 Total Revenue'] = scaler.fit_transform(combined_data[['2019 Total Revenue']])
combined_data['2020 Total Revenue'] = scaler.fit_transform(combined_data[['2020 Total Revenue']])
combined_data['2021 Total Revenue'] = scaler.fit_transform(combined_data[['2021 Total Revenue']])
combined_data['2022 Total Revenue'] = scaler.fit_transform(combined_data[['2022 Total Revenue']])
combined_data['2019 EBITDA'] = scaler.fit_transform(combined_data[['2019 EBITDA']])
combined_data['2020 EBITDA'] = scaler.fit_transform(combined_data[['2020 EBITDA']])
combined_data['2021 EBITDA'] = scaler.fit_transform(combined_data[['2021 EBITDA']])
combined_data['2022 EBITDA'] = scaler.fit_transform(combined_data[['2022 EBITDA']])
combined_data['2019 Net Income'] = scaler.fit_transform(combined_data[['2019 Net Income']])
combined_data['2020 Net Income'] = scaler.fit_transform(combined_data[['2020 Net Income']])
combined_data['2021 Net Income'] = scaler.fit_transform(combined_data[['2021 Net Income']])
combined_data['2022 Net Income'] = scaler.fit_transform(combined_data[['2022 Net Income']])
combined_data['2019 PBT'] = scaler.fit_transform(combined_data[['2019 PBT']])
combined_data['2020 PBT'] = scaler.fit_transform(combined_data[['2020 PBT']])
combined_data['2021 PBT'] = scaler.fit_transform(combined_data[['2021 PBT']])
combined_data['2022 PBT'] = scaler.fit_transform(combined_data[['2022 PBT']])
combined_data['PE Ratio'] = scaler.fit_transform(combined_data[['PE Ratio']])


In [92]:
combined_data.to_csv('companyList_avg_sentiment_clean_news.csv', index=False)

In [93]:
combined_data = combined_data.fillna(1)

In [97]:
combined_data['Avg Sentiment']=((combined_data['Sentiment']*0.3)+(combined_data['News Sentiment Score']*0.7))

# Define the bins and labels for classification
bins = [0,0.6,0.95, 1.1,1.6, 2]
labels = [0, 1, 2,3,4]

# Classify sentiment values into categories
combined_data['Final_Sentiment'] = pd.cut(combined_data['Avg Sentiment'], bins=bins, labels=labels)


combined_data.to_csv('companyList_final_500.csv', index=False,sep=',')


# Using Tpot to create a model

In [105]:
data = pd.read_csv('companyList_final_500_train.csv')

# Separate the features (X) and target variable (y)
X = data[['News Sentiment Score','Entry_point_Sentiment','Growth_Sentiment','Profitability_Sentiment','RedFlag_Sentiment','Performance_Sentiment','Valuation_Sentiment','2019 Total Revenue','2019 EBITDA','2019 Net Income','2019 PBT','2020 Total Revenue','2020 EBITDA','2020 Net Income','2020 PBT','2021 Total Revenue','2021 EBITDA','2021 Net Income','2021 PBT','2022 Total Revenue','2022 EBITDA','2022 Net Income','2022 PBT','PE Ratio']]
y = data['Final_Sentiment']

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape
X_test.shape

# Step 3: Define and run TPOT
tpot = TPOTClassifier(generations=3, population_size=50, verbosity=2, max_time_mins = 5)
tpot.fit(X_train, y_train)

# Step 4: Evaluate TPOT performance
scorer_names = get_scorer_names()
accuracy = tpot.score(X_test, y_test)
print("Accuracy:", accuracy)

# Step 5: Export the optimized pipeline as Python code

tpot.export('tpot_sentiment_pipeline.py')

Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9

Generation 2 - Current best internal CV score: 0.9

Generation 3 - Current best internal CV score: 0.9

Best pipeline: MLPClassifier(input_matrix, alpha=0.1, learning_rate_init=0.01)
Accuracy: 0.85


## copying the tpot_sentiment_pipeline code and using joblib to dump exported_pipeline to a pkl file

In [106]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('companyList_final_500_train.csv', sep=',')
columns_to_drop = ['sNo', 'Company Name','Full Company Name','sc Performance','sc Valuation','sc Growth','sc Profitability','sc Entry Point','sc Red Flags','Income Statements','Sentiment','Avg Sentiment']
tpot_data = tpot_data.drop(columns_to_drop, axis=1)

features = tpot_data.drop('Final_Sentiment', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(tpot_data, tpot_data['Final_Sentiment'], random_state=None)


# Average CV score on the training set was: 0.85
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=True, criterion="gini", max_features=0.55, min_samples_leaf=4, min_samples_split=18, n_estimators=100)),
    MLPClassifier(alpha=0.001, learning_rate_init=0.1)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

import joblib
joblib.dump (exported_pipeline,'Sentiment_Analysis_Flag_Data.pkl')

['Sentiment_Analysis_Flag_Data.pkl']

In [107]:
print(training_features.T)

                               2         66        48        52        98  \
2019 Total Revenue       0.005425  0.002930  0.012108  0.008924  0.004743   
2019 EBITDA              0.279275  0.278094  0.280088  0.289322  0.279269   
2019 Net Income          0.845321  0.845773  0.844544  0.855676  0.846063   
2019 PBT                 0.727548  0.727333  0.727535  0.738577  0.728686   
2020 Total Revenue       0.005585  0.002463  0.009474  0.008342  0.003828   
2020 EBITDA              0.372350  0.368998  0.370031  0.381217  0.370885   
2020 Net Income          1.307394  1.305823  1.306024  1.312670  1.306897   
2020 PBT                 1.074189  1.071928  1.071313  1.081673  1.073315   
2021 Total Revenue       0.006146  0.001413  0.010279  0.008175  0.003551   
2021 EBITDA              0.090071  0.082905  0.085476  0.098268  0.085204   
2021 Net Income          0.953577  0.948644  0.949046  0.957917  0.949729   
2021 PBT                 0.895004  0.888573  0.889350  0.900841  0.890289   

## test the original data in dataframe to test model

In [108]:
testdf = pd.read_csv('companyList_final_500_test.csv', sep=',')
testdf.head()
columns_to_drop = ['sNo', 'Company Name','Full Company Name','sc Performance','sc Valuation','sc Growth','sc Profitability','sc Entry Point','sc Red Flags','Income Statements','Sentiment','Avg Sentiment']
newtestdf = testdf.drop(columns_to_drop, axis=1)
newtestdf.to_csv('Sentiment_Test.csv', index=False)

In [109]:
model = joblib.load(open('Sentiment_Analysis_Flag_Data.pkl', 'rb'))
predictions = pd.DataFrame(model.predict(newtestdf), columns = ['prediction'])
final = pd.concat([predictions, newtestdf], axis = 1)
print(final)
final.to_csv('Sentiment_Test_Prediction.csv', index=False)

     prediction  2019 Total Revenue  2019 EBITDA  2019 Net Income  2019 PBT  \
0             4            0.016452     0.289451         0.856685  0.738837   
1             3            0.033647     0.315758         0.851701  0.732352   
2             4            0.009400     0.283281         0.847774  0.730711   
3             4            0.020859     0.297511         0.864406  0.749209   
4             4            0.023529     0.284738         0.844763  0.727058   
..          ...                 ...          ...              ...       ...   
393           1            0.001395     0.279321         0.847191  0.729498   
394           1            0.000000     0.275234         0.842697  0.724522   
395           1            0.000570     0.276500         0.842959  0.724843   
396           1            0.000000     0.275234         0.842697  0.724522   
397           4            0.000000     0.275234         0.842697  0.724522   

     2020 Total Revenue  2020 EBITDA  2020 Net Inco

In [110]:
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

 

predicted_labels = final['prediction'].values
true_labels = final['Final_Sentiment'].values

 

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)

 

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

 

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[  0   2   0   0   0]
 [  0  77   1   0   0]
 [  0   0 148   0   0]
 [  0   0   6  64   9]
 [  0   0   0   1  90]]
Precision: 0.9493802351754833
Recall: 0.9522613065326633
F1 Score: 0.9485506551857982
